In [1]:
import torch,torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import pandas as pd
import wandb
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor,DefaultTrainer
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from tqdm import tqdm
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import ColorMode
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
PROJECT_NAME = 'Car-Object-Detection-V11-Learning-Detectron2-V2'

In [2]:
data = pd.read_csv('./data.csv')

In [7]:
len(data)

559

In [26]:
info = data.iloc[185]

In [27]:
data['xmin'] = data['xmin'].astype(int)
data['ymin'] = data['ymin'].astype(int)
data['xmax'] = data['xmax'].astype(int)
data['ymax'] = data['ymax'].astype(int)

In [28]:
img = cv2.imread('./data/' + info['image'])
xmin,ymin,xmax,ymax = info['xmin'],info['ymin'],info['xmax'],info['ymax']
x = xmin
y = ymin
w = xmax - xmin
h = ymax - ymin
crop = img[y:y+h,x:x+w]
cv2.imwrite('./crop.png',crop)
cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),2)
cv2.imwrite('./box.png',img)

True

In [29]:
def load_data():
    records = []
    for i in tqdm(range(len(data))):
        record = {}
        info = data.iloc[i]
        path = './data/' + info['image']
        record['file_name'] = path
        record['height'],record['width'] = cv2.imread(path).shape[:2]
        record['image_id'] = i
        record['annotations'] = [
            {
                'bbox':[info['xmin'],info['ymin'],info['xmax'],info['ymax']],
                'bbox_mode':BoxMode.XYXY_ABS,
                'category_id':0
            }
        ]
        records.append(record)
    np.save(records,'./data.npy')
    return records

In [30]:
labels = ['car']

In [31]:
DatasetCatalog.register('data',lambda : load_data())
MetadataCatalog.get('data').set(thing_classes=labels)
metadata = MetadataCatalog.get('data')

In [ ]:
wandb.init(project=PROJECT_NAME,name='baseline')
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(model))
cfg.DATASETS.TRAIN = ('data')
cfg.DATASETS.TEST = ()
cfg.SOLVER.STEPS = []
cfg.SOLVER.MAX_ITER = 500
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(labels)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()
predictor = DefaultPredictor(cfg)
cfg.MODEL.WEIGHTS = './output/model_final.pth'
evaluator = COCOEvaluator('data',output_dir='./output/')
val_loader = build_detection_test_loader(cfg,'data')
metrics = inference_on_dataset(predictor.model,val_loader,evaluator)
wandb.log(metrics)
img = cv2.imread('./data/vid_4_12220.jpg')
preds = predictor(img)
v = Visualizer(img[:,:,::-1],metadata=metadata)
v = v.draw_instance_predictions(preds['instances'].to('cpu'))
v = v.get_image()[:,:,::-1]
plt.figure(figsize=(12,6))
plt.imshow(v)
plt.savefig('./img.png')
plt.close()
wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.finish()

In [ ]:
torch.save(cfg,'cfg.pt')
torch.save(cfg,'cfg.pth')
torch.save(predictor,'predictor.pt')
torch.save(predictor,'predictor.pth')
torch.save(evaluator,'evaluator.pt')
torch.save(evaluator,'evaluator.pth')
torch.save(v,'img.pt')
torch.save(v,'img.pth')
torch.save(model,'model.pt')
torch.save(model,'model.pth')
torch.save(labels,'labels.pt')
torch.save(labels,'labels.pth')
torch.save(metrics,'metrics.pt')
torch.save(metrics,'metrics.pth')